In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/Users/ghostkwebb/Desktop/Plant-Disease/backend/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Dataset path (change if needed)
dataset_path = r"/Users/ghostkwebb/Desktop/archive/CyAUG-Dataset"

# 1. ENHANCED DATA AUGMENTATION FOR THE TRAINING SET
# Create separate data generators: one for training (with augmentation)
# and one for validation (only rescaling).
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # set validation split
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2 # set validation split
)

# Data generators (80% train, 20% validation)
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode="categorical",
    subset="training", # set as training data
    shuffle=True
)

val_generator = validation_datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode="categorical",
    subset="validation", # set as validation data
    shuffle=False # No need to shuffle validation data
)


# 2. DEEPER CNN MODEL WITH DROPOUT FOR REGULARIZATION
model = tf.keras.Sequential([
    # First Conv block
    tf.keras.layers.Conv2D(64, (3,3), activation="relu", input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),

    # Second Conv block
    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),

    # Third Conv block
    tf.keras.layers.Conv2D(256, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),

    # Flatten and Dense layers
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dropout(0.5), # Higher dropout before the final layer
    tf.keras.layers.Dense(len(train_generator.class_indices), activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# 3. INCREASED TRAINING EPOCHS
# Train for more epochs to allow the model to converge.
history = model.fit(train_generator, validation_data=val_generator, epochs=30)

# Save model
model.save("soil_cnn_improved.h5")
print("✅ Model saved as soil_cnn_improved.h5")

Found 4080 images belonging to 7 classes.
Found 1016 images belonging to 7 classes.


/Users/ghostkwebb/Desktop/Plant-Disease/backend/myenv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/ghostkwebb/Desktop/Plant-Disease/backend/myenv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 62s 474ms/step - accuracy: 0.5891 - loss: 1.7687 - val_accuracy: 0.7825 - val_loss: 0.6608
Epoch 2/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 57s 446ms/step - accuracy: 0.8669 - loss: 0.3813 - val_accuracy: 0.8150 - val_loss: 0.6807
Epoch 3/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 57s 443ms/step - accuracy: 0.8980 - loss: 0.2913 - val_accuracy: 0.7854 - val_loss: 0.9792
Epoch 4/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 57s 446ms/step - accuracy: 0.9044 - loss: 0.2716 - val_accuracy: 0.7884 - val_loss: 0.9317
Epoch 5/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 57s 446ms/step - accuracy: 0.9182 - loss: 0.2309 - val_accuracy: 0.7894 - val_loss: 1.1003
Epoch 6/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 59s 464ms/step - accuracy: 0.9297 - loss: 0.2098 - val_accuracy: 0.7510 - val_loss: 1.1030
Epoch 7/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 60s 465ms/step - accuracy: 0.9229 - loss: 0.2084 - val_accuracy: 0.7116 - val_loss: 1.5391
Epoch 8/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 59s 459ms/step - accuracy: 0.9233 - loss: 0

✅ Model saved as soil_cnn_improved.h5
